In [24]:
import pandas as pd
import pyodbc




In [25]:
DB = {'servername': 'LAPTOP-105ECA33\SQLEXPRESS', 
    'database': 'UnitedOutdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor  

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\antho\AppData\Local\Temp\ipykernel_16600\944234878.py:1: SyntaxWarning: invalid escape sequence '\S'
  DB = {'servername': 'LAPTOP-105ECA33\SQLEXPRESS',


In [26]:
export_cursor.execute('USE Northwind')

products_query = "SELECT * From Products"
products_df = pd.read_sql_query(products_query, export_conn)
print(products_df)
export_conn.close()

    ProductID                      ProductName  SupplierID  CategoryID  \
0           1                             Chai           1           1   
1           2                            Chang           1           1   
2           3                    Aniseed Syrup           1           2   
3           4     Chef Anton's Cajun Seasoning           2           2   
4           5           Chef Anton's Gumbo Mix           2           2   
..        ...                              ...         ...         ...   
72         73                       Röd Kaviar          17           8   
73         74                    Longlife Tofu           4           7   
74         75             Rhönbräu Klosterbier          12           1   
75         76                     Lakkalikööri          23           1   
76         77  Original Frankfurter grüne Soße          12           2   

        QuantityPerUnit  UnitPrice  UnitsInStock  UnitsOnOrder  ReorderLevel  \
0    10 boxes x 20 bags      18

C:\Users\antho\AppData\Local\Temp\ipykernel_16600\1704179019.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products_df = pd.read_sql_query(products_query, export_conn)


In [28]:
# Connect to the Microsoft Access database file
conn_str_access = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\antho\OneDrive\Documenten\GitHub\UO_bedrijf\aenc.accdb;'
)

aec_conn = pyodbc.connect(conn_str_access)

# Execute SQL query to select data from the 'product' table in Access
product_query = "SELECT * FROM product"
product_df = pd.read_sql_query(product_query, aec_conn)

print(product_df)

# Close the Access connection
aec_conn.close()


    id          name        description          prod_size   color quantity  \
0  300     Tee Shirt           Tank Top              Small   White       18   
1  301     Tee Shirt             V-neck             Medium  Orange       39   
2  302     Tee Shirt          Crew Neck  One size fits all   Black       72   
3  400  Baseball Cap         Cotton Cap  One size fits all   Black       92   
4  401  Baseball Cap           Wool cap  One size fits all   White       12   
5  500         Visor        Cloth Visor  One size fits all   White       36   
6  501         Visor      Plastic Visor  One size fits all   Black       28   
7  600    Sweatshirt  Hooded Sweatshirt              Large   Green       39   
8  601    Sweatshirt  Zipped Sweatshirt              Large    Blue       32   
9  700        Shorts      Cotton Shorts             Medium   Black       80   

  unit_price picture_name     Category  
0          9  tshirtw.bmp      Clothes  
1         14  tshirto.bmp      Clothes  
2      

C:\Users\antho\AppData\Local\Temp\ipykernel_16600\2144621771.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_df = pd.read_sql_query(product_query, aec_conn)
